In [17]:
from torch.autograd import Variable
import deepchem as dc
from deepchem.feat import Featurizer

from dataloader import *
import model
from os.path import join

# Source code for fc models (tf for classification, tf_regression for regression)
# https://github.com/deepchem/deepchem/blob/master/deepchem/models/tensorgraph/fcnet.py

In [27]:
def benchmark(dir_path, ckpt_file, tasks, _model, featurizer, metric_type):
    
    header = ckpt_file.split('.')[0]
    _ckpt = header+'_'+featurizer
    checkpoint = torch.load(join(dir_path, ckpt_file), map_location=torch.device('cuda'))
    args = checkpoint['args']
    _args = vars(args)
    args.batch_size = 1
    args.test_batch_size = 1
    args.act = 'gelu'
    comet = model.Encoder(args)
    comet.load_state_dict(checkpoint['encoder'])
    comet.eval()

    def mol_to_graph(mol):

        adj = Chem.rdmolops.GetAdjacencyMatrix(mol)
        list_feature = list()
        for atom in mol.GetAtoms():
            list_feature.append(atom_feature(atom))
        return np.array(list_feature), adj

    class MyEncoder(Featurizer):
        name = ['comet_encoder']
        def __init__(self, model):
            self.device = 'cuda'

            self.model = model.to(self.device)

        def _featurize(self, mol):
            X, A = mol_to_graph(mol)
            X = Variable(torch.unsqueeze(torch.from_numpy(X), dim=0)).to(self.device).long()
            A = Variable(torch.unsqueeze(torch.from_numpy(A.astype(float)), dim=0)).to(self.device).float()
            _, _, molvec = self.model(X, A)
            return torch.squeeze(molvec).detach().cpu().numpy()

    if metric_type == 'reg':
        _metric = [dc.metrics.Metric(dc.metrics.mae_score, np.mean)]
    if metric_type == 'cls':
        _metric = [dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)]
    
    if featurizer == 'comet':
        feat = MyEncoder(comet)
    if featurizer == 'raw':
        feat = dc.feat.raw_featurizer.RawFeaturizer()
    if featurizer == 'fingerprint':
        feat = dc.feat.fingerprints.CircularFingerprint(size=256)
    if featurizer == 'default':
        feat = None
    

    task_result = dc.molnet.run_benchmark(
                                 ckpt = _ckpt,
                                 arg = _args,
                                 datasets = tasks,
                                 model = _model,
                                 split = None,
                                 metric = _metric,
                                 n_features = 256,
                                 featurizer = feat,
                                 out_path= './benchmark/',
                                 hyper_parameters = None,
                                 test = True,
                                 reload = False,
                                 seed = 123
                                 )

## task types
---
1) classification: muv, hiv, bace_c, bbbp, tox21, toxcast, sider, clintox


2) regression: lipo, qm7, qm8, delaney, sampl

classification = ['bace_c', 'bbbp', 'clintox', 'hiv', 'muv', 'pcba', 'sider', 'tox21', 'toxcast']
cls_models =  ['rf', 'tf', 'tf_robust', 'logreg', 'irv', 'graphconv', 'dag', 'xgb',
      'weave', 'kernelsvm']

regression = ['bace_r', 'chembl', 'clearance', 'delaney', 'hopv', 'kaggle', 'lipo',
                'nci', 'pdbbind', 'ppb', 'qm7', 'qm7b', 'qm8', 'qm9', 'sampl']
reg_models = ['tf_regression', 'tf_regression_ft', 'rf_regression', 'graphconvreg',
      'dtnn', 'dag_regression', 'xgb_regression', 'weave_regression', 'krr', 'ani']

cls_tasks = ['bace_c', 'bbbp', 'tox21', 'sider', 'clintox']
reg_tasks = ['lipo', 'qm7', 'qm8', 'delaney', 'sampl']

## Fingerprint

In [ ]:
fingerprint_result = dc.molnet.run_benchmark(
                                 ckpt = 'fingerprint',
                                 arg = {'input':'fingerprint'},
                                 datasets = reg_tasks,
                                 model = 'tf_regression',
                                 split = None,
                                 metric =  [dc.metrics.Metric(dc.metrics.mae_score, np.mean)],
                                 n_features = 256,
                                 featurizer = dc.feat.fingerprints.CircularFingerprint(size=256),
                                 out_path= './benchmark/',
                                 hyper_parameters = None,
                                 test = True,
                                 reload = False,
                                 seed = 123
                                 )

In [ ]:
fingerprint_result = dc.molnet.run_benchmark(
                                 ckpt = 'fingerprint',
                                 arg = {'input':'fingerprint'},
                                 datasets = cls_tasks,
                                 model = 'tf',
                                 split = None,
                                 metric = None,
                                 n_features = 256,
                                 featurizer = dc.feat.fingerprints.CircularFingerprint(size=256),
                                 out_path= './benchmark/',
                                 hyper_parameters = None,
                                 test = True,
                                 reload = False,
                                 seed = 123
                                 )

## model_8_256

In [ ]:
model_8_256 = ['model_8_256_001_000004000.tar','model_8_256_005_000020000.tar','model_8_256_010_000040000.tar',
               'model_8_256_015_000060000.tar','model_8_256_020_000080000.tar','model_8_256_025_000100000.tar' ]

In [ ]:
benchmark_models('model_8_256', model_8_256)

In [ ]:
model_4_256 = ['model_4_256_s_no_auxilary_001_000004000.tar', 'model_4_256_s_no_auxilary_002_000008000.tar', 
'model_4_256_s_no_auxilary_003_000012000.tar', 'model_4_256_s_no_auxilary_004_000016000.tar']

In [ ]:
benchmark_models('model_4_256', model_4_256)

In [ ]:
model_4_256_fc = [
'model_4_256_s_fc_001_000004000.tar',
'model_4_256_s_fc_002_000008000.tar',
'model_4_256_s_fc_003_000012000.tar',
'model_4_256_s_fc_004_000016000.tar',
'model_4_256_s_fc_005_000020000.tar',
'model_4_256_s_fc_006_000024000.tar',
'model_4_256_s_fc_007_000028000.tar',
'model_4_256_s_fc_008_000032000.tar']

In [21]:
def cls_benchmark_models(dir_path, files):
    for i, file in enumerate(files):
        benchmark(dir_path, file, ['sider'], 'tf', 'comet', 'cls')
        #benchmark(file, ['qm7'], 'tf_regression', 'comet', 'reg')

In [22]:
def reg_benchmark_models(dir_path, files):
    for i, file in enumerate(files):
        benchmark(dir_path, file, ['qm8'], 'tf_regression', 'comet', 'reg')
        #benchmark(file, ['qm7'], 'tf_regression', 'comet', 'reg')

In [23]:
model_4_256_yeah=[
    'model_4_256_s_fc_022_000088000.tar'
#     'model_4_256_s_fc_016_000064000.tar',
#     'model_4_256_s_fc_018_000072000.tar'
]

In [ ]:
reg_benchmark_models(model_4_256_yeah)

In [28]:
cls_benchmark_models('./runs/model_4_256_s_fc_train', model_4_256_yeah)

-------------------------------------
Benchmark on dataset: sider
-------------------------------------
Loading raw samples now.
shard_size: 8192
About to start loading CSV from /tmp/sider.csv.gz
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
TIMING: featurizing shard 0 took 18.247 s
TIMING: dataset construction took 18.323 s
Loading dataset from disk.
TIMING: dataset construction took 0.073 s
Loading dataset from disk.
TIMING: dataset construction took 0.062 s
Loading dataset from disk.
TIMING: dataset construction took 0.021 s
Loading dataset from disk.
TIMING: dataset construction took 0.019 s
Loading dataset from disk.
computed_metrics: [0.6931176833025262, 0.6517091970200506, 0.8165869218500797, 0.6736416428820966, 0.6640441533236294, 0.6601202099318375, 0.7407132747823727, 0.6865669343080683, 0.6512726145740518, 0.6971654240791318, 0.6717115067848459, 0.6938695683095085, 0.7078650266461244, 0.6813942534149788, 0.6459946652899673, 0.7033216533180777, 0.